In [60]:
import numpy as np
import matplotlib.pyplot as plt

In [61]:
from loaders import clean_text_for_pattern_letter
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [62]:
def predict_train(input_str, letters, max_index:int = 4, max_prev=2):
    symbol_index: dict[str, int] = {}
    SymbolsStats: dict[str, dict] = {}

    for letter in letters:
        symbol_index[letter] = 0

    for inp_idx, inp in enumerate(input_str):
        symbol_index[inp] += 1
        if symbol_index[inp] > max_index:
            symbol_index[inp] = 1

        if inp_idx <= 0: # or inp_idx == len(input_str) - 1:
            continue

        inp_index = symbol_index[inp]
        # print(inp_idx, inp, inp_index)
        
        for rank_idx in range(max_prev):
            prev_idx = inp_idx - rank_idx - 1

            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]
            
            prev_index = symbol_index[prev_inp]

            symbol_with_index = prev_inp + '^' + str(prev_index)

            SymbolsStats.setdefault(symbol_with_index, {})
            SymbolsStats[symbol_with_index].setdefault(rank_idx, {})

            SymbolsStats[symbol_with_index][rank_idx].setdefault(inp, {})
            SymbolsStats[symbol_with_index][rank_idx][inp].setdefault(inp_index, 0)

            SymbolsStats[symbol_with_index][rank_idx][inp][inp_index] += 1
              
    return SymbolsStats

In [63]:
# train
max_index_L1 = 4
max_prev_L1 = 2

SymbolsStats_L1 = predict_train(letters=letters,
                                input_str=input_str,
                                max_index=max_index_L1,
                                max_prev=max_prev_L1,)


In [64]:
sdk_sum = 0
sdk_cnt = 0

for sdk, sdv in SymbolsStats_L1.items():
    print(sdk)
    for sdk_1, sdv_1 in sdv.items():
        print(sdk, sdk_1, sdv_1)
        sdk_sum += len(sdv_1)
        sdk_cnt += 1

    print('')

print(sdk_sum / sdk_cnt)

t^1
t^1 0 {'h': {1: 2}, 'e': {1: 2, 3: 1}, ' ': {1: 1, 4: 1}}
t^1 1 {'e': {1: 1, 4: 1}, 'l': {2: 1, 4: 1}, 'w': {4: 1}, 's': {4: 1}, 'n': {1: 1}}

h^1
h^1 0 {'e': {1: 1, 4: 3}, 'r': {1: 1}, 'a': {4: 1}}
h^1 1 {' ': {1: 2}, 'a': {3: 1}, 'o': {4: 1}, 't': {3: 2}}

e^1
e^1 0 {' ': {1: 2, 4: 1}, 'l': {2: 1, 3: 1}, 'o': {2: 1}, 'd': {2: 1}, '.': {4: 1}, 's': {4: 1, 2: 1}, 'e': {1: 1}, 'k': {2: 1}, 'r': {2: 1}}
e^1 1 {'s': {1: 1}, '.': {1: 1}, 'n': {3: 1}, 'e': {1: 1}, ' ': {3: 2, 4: 2}, 'm': {3: 1}, 'd': {1: 1}, 'k': {2: 1, 3: 1}}

 ^1
 ^1 0 {'s': {1: 1}, 'w': {1: 1}, 't': {1: 1, 2: 1, 4: 1}, 'd': {2: 1, 1: 1}, 'l': {4: 1}, 'h': {1: 1, 2: 1, 3: 1}, 'c': {1: 2, 2: 1}, 'b': {2: 1, 1: 1}, 'a': {3: 2, 2: 1}, 'f': {4: 1, 3: 1}, 'j': {2: 1, 3: 1}}
 ^1 1 {'k': {1: 1}, 'a': {2: 3}, 'e': {1: 2, 4: 3, 3: 1}, ' ': {1: 1}, 'i': {4: 2}, 'h': {1: 2, 4: 1, 2: 1}, 'o': {2: 1, 3: 2}, 'r': {1: 1}}

s^1
s^1 0 {'k': {1: 1}, 'i': {2: 1}, ' ': {1: 2, 2: 1, 3: 1, 4: 1}, 't': {2: 1}}
s^1 1 {'y': {1: 1}, 'n': {2: 1

In [65]:
def predict(input_str, SymbolsStats:dict, max_index, max_rank):
    err = 0
    output = []
    TF = []

    first_inp_index_l = [x for x in range(1, max_index+1)]
    prd_symbol_indices = []
    prd_symbol_indices.append(first_inp_index_l)
    print(prd_symbol_indices[0])

    for inp_idx, inp in enumerate(input_str[:1]):

        predict_per_rank = []

        for rank_idx in range(max_rank):
            prev_idx = inp_idx - rank_idx
            if prev_idx < 0:
                break
            
            prev_inp = input_str[prev_idx]
            
            predict_by_symbol = {}
            print(rank_idx)
            for prev_index in prd_symbol_indices[rank_idx]:
                symbol_with_index = prev_inp + '^' + str(prev_index)
                predict_by_index_symbol = SymbolsStats[symbol_with_index][rank_idx]
                print(predict_by_index_symbol)

                for prd_symbol, prd_index_stats in predict_by_index_symbol.items():
                    has_symbol = predict_by_symbol.get(prd_symbol, 0)

                    if has_symbol == 0:
                        predict_by_symbol.setdefault(prd_symbol, prd_index_stats)
                    else:
                        for index, index_stats in prd_index_stats.items():
                            has_index = predict_by_symbol[prd_symbol].get(index, 0)

                            if has_index == 0:
                                predict_by_symbol[prd_symbol].setdefault(index, index_stats)
                            else:
                                predict_by_symbol[prd_symbol][index] += index_stats
            print()
            print(predict_by_symbol)
            predict_per_rank.append(predict_by_symbol)
        
        print(f'{predict_per_rank=}')
        reduce_ppr = predict_per_rank[0]

        for next_ppr in predict_per_rank[1:]:
            for r_ppr_key, r_ppr in reduce_ppr.items():
                has_symbol = next_ppr.get(r_ppr_key, 0)

                if has_symbol == 0:
                    reduce_ppr.pop(r_ppr_key)
                else:
                    for index, index_stats in r_ppr.items():
                        has_index = next_ppr[r_ppr_key].get(index, 0)

                        if has_index == 0:
                            reduce_ppr[r_ppr_key].pop(r_ppr_key)
                        else:
                            reduce_ppr[r_ppr_key] += index_stats
        
        print(f'{reduce_ppr=}')




In [66]:
predict(input_str=input_str, SymbolsStats=SymbolsStats_L1, max_index=max_index_L1, max_rank=max_prev_L1)

[1, 2, 3, 4]
0
{'h': {1: 2}, 'e': {1: 2, 3: 1}, ' ': {1: 1, 4: 1}}
{'h': {2: 2, 1: 2, 4: 1}, 'u': {1: 1}, '.': {2: 1}}
{' ': {1: 1}, 'o': {3: 1}, 'h': {3: 1}, "'": {3: 1}, 's': {4: 1}, 'w': {1: 1}, 'i': {2: 1}}
{'h': {3: 2, 4: 1}, "'": {1: 1}, 'r': {3: 1}, 'z': {1: 1}, 'o': {2: 1}}

{'h': {1: 4, 2: 2, 4: 2, 3: 3}, 'e': {1: 2, 3: 1}, ' ': {1: 2, 4: 1}, 'u': {1: 1}, '.': {2: 1}, 'o': {3: 1, 2: 1}, "'": {3: 1, 1: 1}, 's': {4: 1}, 'w': {1: 1}, 'i': {2: 1}, 'r': {3: 1}, 'z': {1: 1}}
predict_per_rank=[{'h': {1: 4, 2: 2, 4: 2, 3: 3}, 'e': {1: 2, 3: 1}, ' ': {1: 2, 4: 1}, 'u': {1: 1}, '.': {2: 1}, 'o': {3: 1, 2: 1}, "'": {3: 1, 1: 1}, 's': {4: 1}, 'w': {1: 1}, 'i': {2: 1}, 'r': {3: 1}, 'z': {1: 1}}]
reduce_ppr={'h': {1: 4, 2: 2, 4: 2, 3: 3}, 'e': {1: 2, 3: 1}, ' ': {1: 2, 4: 1}, 'u': {1: 1}, '.': {2: 1}, 'o': {3: 1, 2: 1}, "'": {3: 1, 1: 1}, 's': {4: 1}, 'w': {1: 1}, 'i': {2: 1}, 'r': {3: 1}, 'z': {1: 1}}


In [67]:
def predict_old(input_str, SymbolsStats:dict, max_index, max_prev):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    output = []
    TF = []

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = SymbolsStats.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            # B B {'B': 1} [3] {3: 1} [3] False
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [68]:
# train
max_index_L1 = 4

index_by_prev_L1: dict[str, dict] = {}
StatsDict_L1: dict[str, dict] = {}

StatsDict, index_by_prev_L1 = predict_train(letters=letters, input_str=input_str, max_index=max_index_L1)

ValueError: too many values to unpack (expected 2)

In [ ]:
for sdk, sdv in index_by_prev_L1.items():
    print(sdk, sdv)
    print('')

In [ ]:
for sdk, sdv in StatsDict.items():
    print(sdk, sdv)
    print('')

t^1 {0: {'h^1': 2, 'e^1': 2, ' ^1': 1, ' ^4': 1, 'e^3': 1}, 1: {'e^1': 1, 'l^2': 1, 'l^4': 1, 'e^4': 1, 'w^4': 1, 's^4': 1, 'n^1': 1}}

h^2 {0: {'e^1': 1, 'e^4': 3, 'r^1': 1, 'a^4': 1}, 1: {' ^1': 2, 'a^3': 1, 'o^4': 1, 't^2': 2}}

e^2 {0: {' ^1': 2, 'l^2': 1, 'l^3': 1, 'o^2': 1, 'd^2': 1, ' ^4': 1, '.^4': 1, 's^4': 1, 'k^2': 1, 's^2': 1, 'r^2': 1}, 1: {'s^1': 1, 'e^2': 2, '.^1': 1, 'n^3': 1, ' ^3': 2, 'm^3': 1, 'd^1': 1, ' ^4': 2, 'k^2': 1, 'k^3': 1}}

 ^2 {0: {'s^1': 1, 'w^1': 1, 't^4': 1, 'd^2': 1, 'l^4': 1, 'h^1': 1, 'h^2': 1, 't^1': 1, 't^3': 1, 'c^1': 2, 'b^2': 1, 'd^1': 1, 'a^3': 2, 'a^2': 1, 'f^4': 1, 'c^2': 1, 'h^3': 1, 'j^2': 1, 'b^1': 1, 'j^3': 1, 'f^3': 1}, 1: {'k^1': 1, 'a^2': 3, 'e^1': 2, 'e^4': 3, 'i^4': 2, 'e^3': 1, 'h^1': 2, 'h^4': 1, 'h^2': 1, 'o^2': 1, 'r^1': 1, ' ^2': 3, 'o^3': 2}}

s^2 {0: {'k^1': 1, 'i^2': 1, ' ^1': 2, ' ^2': 1, ' ^3': 1, ' ^4': 1, 't^1': 1}, 1: {'y^1': 1, 'n^2': 1, 'h^2': 1, 'd^3': 1, 'a^3': 1, 'a^2': 1, 'i^3': 1, 'h^1': 1}}

k^2 {0: {'y^1': 1, '

In [ ]:
# predict
TF = []

TF, err, out_L1 = predict(input_str=input_str, TF=TF, SymbolsStats=StatsDict, index_by_prev=index_by_prev_L1, max_index=max_index_L1)

print(err, err / (len(input_str) / 100) )

ValueError: max() arg is an empty sequence

In [ ]:
TF = np.array(TF, dtype=float)
# print(TF)
TF = np.pad(TF, (1, 1), constant_values=[1,1])
# print(TF)
TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (1, 0))[:-1] 
# TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (0, 1))[-1:]
# print(TF)
TF[np.where(TF < 0)] = 0
# TF = np.abs(TF)
TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF[np.where(TF <= 0.5)] = 0
# print(TF.size, TF.max(), TF.min())

# fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
# # axs.plot(np.arange(TF.size), TF)
# markerline, stemline, baseline, = axs.stem(np.arange(TF.size), TF)
# plt.setp(markerline, markersize = 1)
# axs.xaxis.set_ticks(np.arange(len(input_str)) ,input_str)
# # axs.xaxis.grid(True)

# plt.show()

In [ ]:
upstream_inp: dict[int, list] = {}

temp_seq = []

for inp_idx, inp in enumerate(input_str):
    tf = TF[inp_idx]
    
    if tf > 0:
        upstream_inp.setdefault(inp_idx, temp_seq.copy())
        temp_seq.clear()

    temp_seq.append(inp)

for item, val in upstream_inp.items():
    print(item, val)
    
print(len(upstream_inp))

5 ['t', 'h', 'e', ' ', 's']
18 ['k', 'y', ' ', 'a', 'b', 'o', 'v', 'e', ' ', 't', 'h', 'e', ' ']
32 ['p', 'o', 'r', 't', ' ', 'w', 'a', 's', ' ', 't', 'h', 'e', ' ', 'c']
37 ['o', 'l', 'o', 'r', ' ']
53 ['o', 'f', ' ', 't', 'e', 'l', 'e', 'v', 'i', 's', 'i', 'o', 'n', ',', ' ', 't']
59 ['u', 'n', 'e', 'd', ' ', 't']
66 ['o', ' ', 'a', ' ', 'd', 'e', 'a']
72 ['d', ' ', 'c', 'h', 'a', 'n']
116 ['n', 'e', 'l', '.', ' ', '"', 'i', 't', "'", 's', ' ', 'n', 'o', 't', ' ', 'l', 'i', 'k', 'e', ' ', 'i', "'", 'm', ' ', 'u', 's', 'i', 'n', 'g', ',', '"', ' ', 'c', 'a', 's', 'e', ' ', 'h', 'e', 'a', 'r', 'd', ' ', 's']
124 ['o', 'm', 'e', 'o', 'n', 'e', ' ', 's']
135 ['a', 'y', ',', ' ', 'a', 's', ' ', 'h', 'e', ' ', 's']
155 ['h', 'o', 'u', 'l', 'd', 'e', 'r', 'e', 'd', ' ', 'h', 'i', 's', ' ', 'w', 'a', 'y', ' ', 't', 'h']
166 ['r', 'o', 'u', 'g', 'h', ' ', 't', 'h', 'e', ' ', 'c']
175 ['r', 'o', 'w', 'd', ' ', 'a', 'r', 'o', 'u']
183 ['n', 'd', ' ', 't', 'h', 'e', ' ', 'd']
187 ['o', 'o', 'r',

In [ ]:
letters_dict: dict[str, int] = {}
for let_idx, letter in enumerate(letters):
    letters_dict.setdefault(letter, let_idx)

upstream_patterns = []

for upstream_letters in upstream_inp.values():
    upstream_pattern = np.zeros(len(letters))

    for letter in upstream_letters:
        pattern_idx = letters_dict.get(letter, 0)

        upstream_pattern[pattern_idx] = 1
    
    upstream_patterns.append(upstream_pattern)

upstream_patterns = np.array(upstream_patterns)

for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [ ]:
layer_size = 16

I = np.zeros(upstream_patterns.shape[1])
O = np.zeros(layer_size)
W = np.ones((O.size, I.size))
Trace = np.zeros((O.size, I.size))
print('W', W.shape, W.max(), W.min(), np.median(W))

k = 1
push_delta = 0.4

prec = 1e-30

epoch_cnt = 100
eps0 = 0.02

eps = 0.05
rate_drop = 0.0

AVG_O = np.zeros(O.shape)
AVG_O_CNT = np.zeros(O.shape)

samples_idx = np.arange(upstream_patterns.shape[0])

for epoch_idx in range(epoch_cnt):
    np.random.shuffle(samples_idx)

    for i_idx in samples_idx:
        I = upstream_patterns[i_idx]

        # infer
        O = np.dot(W, I)

        # train
        winner_ids = np.argsort(O)[::-1][:1+k]
        winner = winner_ids[0]
        loosers = winner_ids[1:1+k]

        # update
        W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
        W[winner] = np.clip(W[winner], -1, 1)

        # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
        # W[loosers] = np.clip(W[loosers], -1, 1)

        # update Trace
        rate_Trace = np.zeros(I.shape)
        rate_Trace[(I > 0.5)] = 0.3
        rate_Trace[(I < 0.5)] = -0.001

        dTrace = np.zeros(W.shape)
        dTrace[winner] = rate_Trace
        Trace += dTrace
        Trace = np.clip(Trace, 0, 1)

        eps -= eps * rate_drop

print('W', W.shape, W.max(), W.min(), np.median(W))

W (16, 32) 1.0 1.0 1.0
W (16, 32) 1.0 -0.49999999999991207 -0.37695569228511816


In [ ]:
SL_symbols = [x.capitalize() for x in letters][:layer_size]
print(len(SL_symbols))

SLSymbolDict = {}

SL_symbols_d = {}
for idx, val in enumerate(SL_symbols):
    SL_symbols_d.setdefault(val + ':' + str(idx), 0)

upstream_inp_keys = list(upstream_inp.keys())

upstream_cluster_by_symbol: dict[str, int] = {}
for idx, val in enumerate(SL_symbols):
    upstream_cluster_by_symbol.setdefault(val, idx)

for upstr_idx, up_ptr in enumerate(upstream_patterns):
    O = np.dot(W, up_ptr)

    winner = np.argmax(O)
    symbol = SL_symbols[winner]
    print(SL_symbols[winner])

    # print(W[upstream_cluster_by_symbol[symbol]].tolist())

    sls = SL_symbols[winner]
    sls_key = upstream_inp_keys[upstr_idx]
    SLSymbolDict.setdefault(sls_key, sls)
    
    SL_symbols_d[SL_symbols[winner] + ':' + str(winner)] += 1

print(len(SL_symbols_d))
for key, val in SL_symbols_d.items():
    print(key, val)

print(SLSymbolDict)

16
J
J
E
I
I
K
J
C
J
O
J
I
G
F
B
F
A
J
C
E
I
K
I
L
E
I
E
M
J
H
D
N
P
C
16
A:0 1
B:1 1
C:2 3
D:3 1
E:4 4
F:5 2
G:6 1
H:7 1
I:8 6
J:9 7
K:10 2
L:11 1
M:12 1
N:13 1
O:14 1
P:15 1
{5: 'J', 18: 'J', 32: 'E', 37: 'I', 53: 'I', 59: 'K', 66: 'J', 72: 'C', 116: 'J', 124: 'O', 135: 'J', 155: 'I', 166: 'G', 175: 'F', 183: 'B', 187: 'F', 198: 'A', 237: 'J', 266: 'C', 311: 'E', 355: 'I', 375: 'K', 384: 'I', 392: 'L', 404: 'E', 415: 'I', 429: 'E', 436: 'M', 455: 'J', 459: 'H', 465: 'D', 468: 'N', 484: 'P', 495: 'C'}


In [ ]:
sl_input = list(SLSymbolDict.values())
print(sl_input)

['J', 'J', 'E', 'I', 'I', 'K', 'J', 'C', 'J', 'O', 'J', 'I', 'G', 'F', 'B', 'F', 'A', 'J', 'C', 'E', 'I', 'K', 'I', 'L', 'E', 'I', 'E', 'M', 'J', 'H', 'D', 'N', 'P', 'C']


In [ ]:
# SL train
max_index_L2 = 4

StatsDict_L2, index_by_prev_L2 = predict_train(input_str=sl_input, letters=SL_symbols, max_index=max_index_L2)

In [ ]:
for sdk, sdv in StatsDict_L2.items():
    print(sdk, sdv)
    print('')

J1:J1 {'E': 1}

J2:E1 {'I': 1}

E1:I1 {'I': 1}

I1:I1 {'K': 1}

I2:K1 {'J': 1}

K1:J3 {'C': 1}

J3:C1 {'J': 1}

C1:J4 {'O': 1}

J4:O1 {'J': 1}

O1:J1 {'I': 1}

J1:I3 {'G': 1}

I3:G1 {'F': 1}

G1:F1 {'B': 1}

F1:B1 {'F': 1}

B1:F2 {'A': 1}

F2:A1 {'J': 1}

A1:J2 {'C': 1}

J2:C2 {'E': 1}

C2:E2 {'I': 1}

E2:I4 {'K': 1}

I4:K2 {'I': 1}

K2:I1 {'L': 1}

I1:L1 {'E': 1}

L1:E3 {'I': 1}

E3:I2 {'E': 1}

I2:E4 {'M': 1}

E4:M1 {'J': 1}

M1:J3 {'H': 1}

J3:H1 {'D': 1}

H1:D1 {'N': 1}

D1:N1 {'P': 1}

N1:P1 {'C': 1}



In [ ]:
for sdk, sdv in index_by_prev_L2.items():
    print(sdk, sdv)
    print('')

J1:J {1: 1}

J0:J {1: 1}

J2:E {1: 1}

J0:E {1: 1}

E1:I {1: 1}

E0:I {1: 1, 4: 1, 2: 1}

I1:I {1: 1}

I0:I {1: 1}

I2:K {1: 1}

I0:K {1: 1, 2: 1}

K1:J {3: 1}

K0:J {3: 1}

J3:C {1: 1}

J0:C {1: 1, 2: 1}

C1:J {4: 1}

C0:J {4: 1}

J4:O {1: 1}

J0:O {1: 1}

O1:J {1: 1}

O0:J {1: 1}

J1:I {3: 1}

J0:I {3: 1}

I3:G {1: 1}

I0:G {1: 1}

G1:F {1: 1}

G0:F {1: 1}

F1:B {1: 1}

F0:B {1: 1}

B1:F {2: 1}

B0:F {2: 1}

F2:A {1: 1}

F0:A {1: 1}

A1:J {2: 1}

A0:J {2: 1}

J2:C {2: 1}

C2:E {2: 1}

C0:E {2: 1}

E2:I {4: 1}

I4:K {2: 1}

K2:I {1: 1}

K0:I {1: 1}

I1:L {1: 1}

I0:L {1: 1}

L1:E {3: 1}

L0:E {3: 1}

E3:I {2: 1}

I2:E {4: 1}

I0:E {4: 1}

E4:M {1: 1}

E0:M {1: 1}

M1:J {3: 1}

M0:J {3: 1}

J3:H {1: 1}

J0:H {1: 1}

H1:D {1: 1}

H0:D {1: 1}

D1:N {1: 1}

D0:N {1: 1}

N1:P {1: 1}

N0:P {1: 1}



In [ ]:
# predict
TF_L2 = []

TF_L2, err_L2, out_L2 = predict(input_str=sl_input, 
                                TF=TF_L2, 
                                SymbolsStats=StatsDict_L2, 
                                index_by_prev=index_by_prev_L2, 
                                max_index=max_index_L2)

for o in out_L2:
    print(o)

print(err_L2, err_L2 / (len(sl_input) / 100), len(sl_input), len(out_L2) )

E E {'E': 1} [1] {2: 1} [1, 2, 3, 4] True
I I {'I': 1} [1] {1: 1} [2] False
I I {'I': 1} [1] {1: 1} [1] False
K K {'K': 1} [1] {2: 1} [1] True
J J {'J': 1} [1] {1: 1} [2] False
C C {'C': 1} [3] {3: 1} [1] False
J J {'J': 1} [1] {1: 1} [3] False
O O {'O': 1} [4] {4: 1} [1] False
J J {'J': 1} [1] {1: 1} [4] False
I I {'I': 1} [1] {1: 1} [1] False
G G {'G': 1} [3] {3: 1} [1] False
F F {'F': 1} [1] {1: 1} [3] False
B B {'B': 1} [1] {1: 1} [1] False
F F {'F': 1} [1] {1: 1} [1] False
A A {'A': 1} [2] {2: 1} [1] False
J J {'J': 1} [1] {1: 1} [2] False
C C {'C': 1} [2] {2: 1} [1] False
E E {'E': 1} [2] {2: 1} [2] False
I I {'I': 1} [2] {2: 1} [2] False
K K {'K': 1} [4] {4: 1} [2] False
I I {'I': 1} [2] {2: 1} [4] False
L L {'L': 1} [1] {1: 1} [2] False
E E {'E': 1} [1] {1: 1} [1] False
I I {'I': 1} [3] {3: 1} [1] False
E E {'E': 1} [2] {2: 1} [3] False
M M {'M': 1} [4] {4: 1} [2] False
J J {'J': 1} [1] {1: 1} [4] False
H H {'H': 1} [3] {3: 1} [1] False
D D {'D': 1} [1] {1: 1} [3] False
N N {'N

In [ ]:
# from itertools import cycle

def ds_predict(input_str, 
               StatsDict: dict, 
               index_by_prev: dict,
               upstream_cluster_by_symbol: dict,
               letters_dict: dict,
               out_L2: list,
               SLSymbolDict: dict,
               W,
               max_index):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    TF = []
    output = []

    ds_out_idx = 0
    ds_inp = '0'
    ds_pattern = np.ones(W.shape[1])

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        # downstream pattern
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
                ds_pattern = np.ones(W.shape[1])
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_pattern = W[upstream_cluster_by_symbol.get(ds_inp)]
                # ds_pattern = np.copy(W[upstream_cluster_by_symbol.get(ds_inp)])
                # ds_pattern[np.where(ds_pattern < 0)] = 0

                ds_out_idx += 1
        print(stats_d)
        for st_key, st in stats_d.items():
            letter_idx = letters_dict.get(st_key)
            
            ds_stats = ds_pattern[letter_idx]
            print(st_key, ds_stats)
            if ds_stats <= 0:
                stats_d[st_key] = 0

                continue
            
            stats_d[st_key] *= ds_stats
        
        print(stats_d)
        print(input_str[inp_idx + 1])
        print()

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [ ]:
dsp_TF, dsp_err, dsp_output = ds_predict(input_str=input_str,
                                         StatsDict=StatsDict,
                                         index_by_prev=index_by_prev_L1,
                                         upstream_cluster_by_symbol=upstream_cluster_by_symbol,
                                         letters_dict=letters_dict,
                                         out_L2=out_L2,
                                         SLSymbolDict=SLSymbolDict,
                                         W=W,
                                         max_index=max_index_L1)

print(dsp_err, dsp_err / (len(input_str) / 100))


{'e': 4}
e 1.0
{'e': 4.0}
e

{' ': 4, 'a': 1, 't': 1, 'r': 1}
  1.0
a 1.0
t 1.0
r 1.0
{' ': 4.0, 'a': 1.0, 't': 1.0, 'r': 1.0}
 

{'s': 2, 'd': 2, 'm': 1, 'c': 1}
s 1.0
d 1.0
m 1.0
c 1.0
{'s': 2.0, 'd': 2.0, 'm': 1.0, 'c': 1.0}
s

{'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1}
k 1.0
a 1.0
o 1.0
p 1.0
h 1.0
{'k': 1.0, 'a': 1.0, 'o': 1.0, 'p': 2.0, 'h': 1.0}
k

{'y': 1}
y 0.946900806318342
{'y': 0.946900806318342}
y

{' ': 1}
  1.0
{' ': 1.0}
 

{'a': 1}
a 1.0
{'a': 1.0}
a

{'b': 1}
b 0.9666379108512058
{'b': 0.9666379108512058}
b

{'o': 1}
o 1.0
{'o': 1.0}
o

{'v': 1}
v 0.8074313728335321
{'v': 0.8074313728335321}
v

{'e': 1}
e 1.0
{'e': 1.0}
e

{' ': 1}
  1.0
{' ': 1.0}
 

{'t': 1, 's': 1, 'c': 2, 'i': 1, 'f': 1}
t 0.9976816656746194
s 1.0
c 0.6638427376423213
i 0.9826838848386042
f -0.49999999999983735
{'t': 0.9976816656746194, 's': 1.0, 'c': 1.3276854752846425, 'i': 0.9826838848386042, 'f': 0}
t

{'h': 2, 'o': 1, 'e': 1}
h 1.0
o 1.0
e 1.0
{'h': 2.0, 'o': 1.0, 'e': 1.0}
h

{'e': 4, 'r': 1}


In [ ]:
def downstream_predict_train(input_str: str | list,
                             sl_input: list,
                             SLSymbolDict: dict,
                            #  ds_out: list,
                             StatsDict_L1: dict,
                            #  StatsDict_L2,
                            #  TF_L1,
                             index_by_prev_L1: dict,
                            #  index_by_prev_L2,
                             max_index_L1: int,
                            #  max_index_L2,
                             ):
    next_index: dict[str, int] = {}

    for letter in letters:
        next_index[letter] = 1

    ds_inp = '0'
    sl_inp_idx = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if sl_inp_idx == len(sl_input):
                ds_inp = '0'
            else:
                ds_inp = sl_input[sl_inp_idx]
                sl_inp_idx += 1
        # print(f'{ds_inp=}')
        stats_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict_L1.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict_L1[stats_key].setdefault(next_inp, 0)

        StatsDict_L1[stats_key][next_inp] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(0) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index_L1:
            next_index[prev_inp] = 1
        
    return StatsDict_L1, index_by_prev_L1


In [ ]:
StatsDict_L1_ds = {}
index_by_prev_L1_ds = {}

StatsDict_L1_ds, index_by_prev_L1_ds = downstream_predict_train(input_str=input_str,
                                                                sl_input=sl_input,
                                                                SLSymbolDict=SLSymbolDict,
                                                                StatsDict_L1=StatsDict_L1_ds,
                                                                index_by_prev_L1=index_by_prev_L1_ds,
                                                                max_index_L1=max_index_L1, )

for key, val in StatsDict_L1_ds.items():
    print(key, val)

0:t1:h1 {'e': 1}
0:h1:e1 {' ': 1}
0:e1: 1 {'s': 1}
0: 1:s1 {'k': 1}
J:s1:k1 {'y': 1}
J:k1:y1 {' ': 1}
J:y1: 2 {'a': 1}
J: 2:a1 {'b': 1}
J:a1:b1 {'o': 1}
J:b1:o1 {'v': 1}
J:o1:v1 {'e': 1}
J:v1:e2 {' ': 1}
J:e2: 3 {'t': 1}
J: 3:t2 {'h': 1}
J:t2:h2 {'e': 1}
J:h2:e3 {' ': 2}
J:e3: 4 {'p': 1}
E: 4:p1 {'o': 1}
E:p1:o2 {'r': 1}
E:o2:r1 {'t': 1}
E:r1:t3 {' ': 1}
E:t3: 1 {'w': 1}
E: 1:w1 {'a': 1}
E:w1:a2 {'s': 1}
E:a2:s2 {' ': 1}
E:s2: 2 {'t': 1}
E: 2:t4 {'h': 1}
E:t4:h3 {'e': 1}
E:h3:e4 {' ': 1, 'a': 1}
E:e4: 3 {'c': 1}
E: 3:c1 {'o': 1}
I:c1:o3 {'l': 1}
I:o3:l1 {'o': 1}
I:l1:o4 {'r': 1}
I:o4:r2 {' ': 2}
I:r2: 4 {'o': 1}
I: 4:o1 {'f': 1}
I:o1:f1 {' ': 1}
I:f1: 1 {'t': 1}
I: 1:t1 {'e': 1}
I:t1:e1 {'l': 1, 's': 1}
I:e1:l2 {'e': 1}
I:l2:e2 {'v': 1}
I:e2:v2 {'i': 1}
I:v2:i1 {'s': 1}
I:i1:s3 {'i': 1}
I:s3:i2 {'o': 1}
I:i2:o2 {'n': 1}
I:o2:n1 {',': 1}
I:n1:,1 {' ': 1}
I:,1: 2 {'t': 1}
I: 2:t2 {'u': 1}
K:t2:u1 {'n': 1}
K:u1:n2 {'e': 1}
K:n2:e3 {'d': 1}
K:e3:d1 {' ': 1}
K:d1: 3 {'t': 1}
K: 3:t3 {'o': 1

In [ ]:
from itertools import cycle

print(out_L2)
def downstream_predict(input_str: str | list,
                       SLSymbolDict: dict,
                       StatsDict_L1_ds: dict,
                       index_by_prev_L1_ds: dict,
                       out_L2: list,
                       max_index_L1: int,
                       TF: list,
                       err=0,):
    prev_index_l = [x for x in range(1, max_index_L1+1)]

    ds_inp = '0'
    ds_out_idx = 0
    ds_predict_out = cycle(out_L2)

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]

        # print(inp, SLSymbolDict.get(inp_idx, 0))
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_out_idx += 1
        print(ds_inp, is_peak)
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev_L1_ds.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = ds_inp + ':' + prev_inp + \
                        str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict_L1_ds.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index_L1:
                max_key_d = 1

            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)

        prev_index_l = list(index_d.keys())

        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)

    return TF, err


['J', 'E', 'I', 'I', 'K', 'J', 'C', 'J', 'O', 'J', 'I', 'G', 'F', 'B', 'F', 'A', 'J', 'C', 'E', 'I', 'K', 'I', 'L', 'E', 'I', 'E', 'M', 'J', 'H', 'D', 'N', 'P', 'C']


In [ ]:
TF_L1_ds = []

TF_L1_ds, err_L1_ds = downstream_predict(input_str=input_str,
                                         SLSymbolDict=SLSymbolDict,
                                         StatsDict_L1_ds=StatsDict_L1_ds,
                                         index_by_prev_L1_ds=index_by_prev_L1_ds,
                                         out_L2=out_L2,
                                         max_index_L1=max_index_L1,
                                         TF=TF_L1_ds, )

print(err_L1_ds, err_L1_ds / (len(input_str) / 100))

0 0
0 0
0 0
0 0
J J
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
E J
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
E 0
I E
I 0
I 0
I 0
I 0
I I
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
K I
K 0
K 0
K 0
K 0
K 0
J K
J 0
J 0
J 0
J 0
J 0
J 0
C J
C 0
C 0
C 0
C 0
C 0
J C
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
O J
O 0
O 0
O 0
O 0
O 0
O 0
O 0
J O
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
I J
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
I 0
G I
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
F G
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
B F
B 0
B 0
B 0
B 0
B 0
B 0
B 0
F B
F 0
F 0
F 0
A F
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
A 0
J A
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
J 0
C J
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
C 0
